pyarrowで書き込み→daskで読み込んだときに謎のエラーが出るか再現を試みる。

In [1]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# 動作環境の確認
print(pd.__version__)
# print(np.__version__)
print(pa.__version__)

1.1.2
11.0.0


In [3]:
df = pd.DataFrame({
    'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
    'number'    : [3, 2, 4, 3, 2, 1],
})

In [4]:
# pandas dataframeから pyarrow tableに変換
table = pa.Table.from_pandas(df)

In [5]:
pq.write_table(table, "output.parquet")

In [6]:
# daskで読み込む

ddf = dd.read_parquet("output.parquet")

ここまでは上手く行っている。

In [7]:
for i in range(3):
    df = pd.DataFrame({
        'name'    : ['Alice', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
        'item' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff'],
        'number'    : [i * 10] * 6,
    })
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f"dir1/output{i}.parquet")
    

In [8]:
ddf = dd.read_parquet("dir1/*.parquet")

In [9]:
ddf

,name,item,number
npartitions=3,,,
,object,object,int64
,...,...,...
,...,...,...
,...,...,...


In [10]:
ddf.compute()

,name,item,number
0,Alice,aaa,0
1,Bob,bbb,0
2,Charlie,ccc,0
3,Charlie,ddd,0
4,Alice,eee,0
5,Bob,fff,0
0,Alice,aaa,10
1,Bob,bbb,10
2,Charlie,ccc,10
3,Charlie,ddd,10


別にエラーは出なかったな。何がまずかったのだろう…